#### **BigBasket Product Query Engine  Notebook 1**
**It contains**
1. smooth data transformation 
2. extracted important meta data,  encoding done using sentence encoder and stored in dict object along with meta data  
3. returns space.pkl which will use to store context in vector db(qdrant) file containg vector representation of context and important meta data

In [ ]:
!pip install -q qdrant-client
!pip install -q sentence-transformers
!pip install -q torch

In [ ]:
import torch
import pandas as pd
import sys
import json
import warnings
warnings.filterwarnings('ignore')
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models
from tqdm.auto import tqdm
from typing import List



In [ ]:
CSV_PATH =  "/kaggle/input/chaabi/bigBasketProducts.csv"
df = pd.read_csv(CSV_PATH).drop(columns = ["index"])

In [ ]:
df

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
    
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    retriever = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)
# retriever = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)
retriever

In [ ]:
class make_embedding_ds(torch.utils.data.IterableDataset):
    def __init__(self, csv = df):
        """
        will compute embedding using sentence encoder
        
        """
        super(make_embedding_ds).__init__()
        self.csv = csv
        self.total_row = csv.shape[0]
        self.col = csv.columns.to_list()

    def __iter__(self):
        for row_no in range(self.total_row):
            
            #testing: make comment it later
       
            
            
            row = self.csv.iloc[row_no].to_dict()
            product_name = row[self.col[0]]
            story = f"{row[ self.col[0] ]} is of category {row[self.col[1]]} and sub category is {row[self.col[2]]}. {row[self.col[0]]} is type {row[self.col[6]]}. brand of {row[self.col[0]]} is {row[self.col[3]]}, with rating {row[self.col[7]]}. sale price of {row[self.col[0]]} is {row[self.col[4]]} with market price {row[self.col[5]]}, description of {row[self.col[0]]} is {row[self.col[8]]}"
            emb = retriever.encode(story)
            
            
            yield row_no,product_name,story,emb
    

ds = make_embedding_ds(df)


dataloader = torch.utils.data.DataLoader(ds, num_workers=1,batch_size=128)


In [ ]:
json_emb = {
    "payload":[],
    "emb": []
}
batch_no = 1
for row,product_name,story,emb in dataloader:
    
    _batch_len = len(row)
    
    
    
    print(f"batch no-{batch_no} competed")
    batch_no += 1
    
    for index in range(_batch_len):
        
        temp = {}
        temp["id"] = row[index].item()
        temp["product"] = product_name[index]
        temp["story"] = story[index]
        
        json_emb["payload"].append(temp)
        json_emb["emb"].append(emb[index].numpy().tolist())
        
    

    
#     json_emb["payload"].append(batch[0])
#     json_emb["emb"].append(batch[1][0])

    


In [ ]:
sys.getsizeof(json_emb)/1024

In [ ]:
import pickle

# Deserialize the data from the pickle file
with open('space.pkl', 'wb') as file:
    pickle.dump(json_emb, file)